In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [2]:
# Load the saved tensors
# loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt")

# # Access the tensors
# index = loaded_data1['index']
# images = loaded_data1['images']
# labels = loaded_data1['labels']
# #print shape of tensors
# images = images / 255
#     # mean normalize
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
# print(images.shape)
# print(labels.shape)
# train_images=images[:12250]
# train_labels=labels
# #print the labels
# print(labels)
# #count the number of unique labels
# # count the number of 1s and 0s
# unique, counts = np.unique(labels, return_counts=True)
# print(dict(zip(unique, counts)))

# # subset_labels = labels[:17000]   
# # num_ones = np.count_nonzero(subset_labels == 1)
# # num_zeros = np.count_nonzero(subset_labels == 0)

# # print(f"Number of 1s(train ): {num_ones}")
# # print(f"Number of 0s(train): {num_zeros}")

In [3]:
# import torchvision.transforms as transforms

# s=0.5
# color_jitter = transforms.ColorJitter(0.4*4, 0.4*s, 0.4*s, 0.2*s)

# transform_eval = transforms.Compose([
#         # transforms.RandomResizedCrop((224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomApply([color_jitter], p=0.5), 
#         transforms.RandomGrayscale(p=0.4),
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#         ])

# train_images = transform_eval(train_images)
# train_dataset = TensorDataset(images, labels)
# train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

In [4]:
# Load the saved tensors
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")

# Access the tensors
index = loaded_data['index']
images1 = loaded_data['images']
labels1 = loaded_data['labels']
images1=images1
labels1=labels1
#print shape of tensors
images1 = images1 / 255
    # mean normalize
images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
print(images1.shape)
print(labels1.shape)
#print the labels
print(labels1)
#count the number of unique labels
# count the number of 1s and 0s
unique, counts = np.unique(labels1, return_counts=True)
print(dict(zip(unique, counts)))

subset_labels = labels1  
num_ones = np.count_nonzero(subset_labels == 1)
num_zeros = np.count_nonzero(subset_labels == 0)

print(f"Number of 1s(train ): {num_ones}")
print(f"Number of 0s(train): {num_zeros}")

torch.Size([10025, 3, 224, 224])
torch.Size([10025])
tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8)
{0: 8983, 1: 1042}
Number of 1s(train ): 1042
Number of 0s(train): 8983


In [5]:

test_dataset = TensorDataset(images1, labels1)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
# train_model = EfficientNetClassifier(
#     models.efficientnet_b0,EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
# ).to(device)

# iter_losses, epoch_losses = train_fn(
#     train_model,
#     nn.CrossEntropyLoss(),
#     dataloader=train_loader,
#     lr=3e-4,
#     epochs=50,
#     verbose=True,
#     wandb_log=False,
# )

In [9]:
import torchvision.models as models

efficientnet = models.efficientnet_b0(pretrained=True)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

efficientnet.classifier[1]=nn.Linear(in_features=1280, out_features=2, bias=True)

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(


In [ ]:
model = efficientnet.to(device) 

In [11]:
model.load_state_dict(torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/strafied_efficient_whole_bangladesh.pth"))

<All keys matched successfully>

In [12]:
with torch.no_grad():
    model.eval()
    true_labels = []
    pred_labels = []
    for i, (inputs, labels) in enumerate(test_loader):
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        output = model(inputs)
        _, preds = torch.max(output, 1)
        true_labels.extend(labels.data.cpu().numpy())
        pred_labels.extend(preds.data.cpu().numpy())
test_accuracy = accuracy_score(true_labels, pred_labels)
# test_accuracies.append(test_accuracy)
test_precision = precision_score(true_labels, pred_labels)
# test_precisions.append(test_precision)
test_recall = recall_score(true_labels, pred_labels)
# test_recalls.append(test_recall)
test_f1 = f1_score(true_labels, pred_labels)
# test_f1_scores.append(test_f1)

In [13]:
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")

Test Accuracy: 0.9265835411471322
Test Precision: 0.9090909090909091
Test Recall: 0.32629558541266795
Test F1 Score: 0.48022598870056493


: 

In [ ]:
with torch.no_grad():
    pred_classes =train_model.predict_class(
        dataloader=test_loader, batch_size=512, verbose=True
    ).to(device)

test_labels = labels.to(device)
print("Accuracy: ", accuracy_score(pred_classes,test_labels))
print("Precision: ", precision_score(pred_classes,test_labels))
print("Recall: ", recall_score(pred_classes,test_labels))
print("F1: ", f1_score(pred_classes,test_labels))
print("classification report: ", classification_report(pred_classes,test_labels))

100%|██████████| 50/50 [01:13<00:00,  1.48s/it]


Accuracy:  tensor(0.8760, device='cuda:0')
Precision:  tensor(0.0688, device='cuda:0')
Recall:  tensor(0.0689, device='cuda:0')
F1:  tensor(0.0689, device='cuda:0')
classification report:  {'accuracy': tensor(0.8760, device='cuda:0'), 'precision': tensor(0.0688, device='cuda:0'), 'recall': tensor(0.0689, device='cuda:0'), 'f1': tensor(0.0689, device='cuda:0')}


In [ ]:
y_pred = pred_classes.cpu().numpy()
